In [1]:
import os
os.environ["JAVA_HOME"] = "/mnt/extproj/projekte/textmining/jdk/openlogic-openjdk-11.0.22+7-linux-x64"


In [2]:
from collections import defaultdict
from pyspark.sql import SparkSession

from pyspark.sql.functions import *
from pyspark.sql.types import *

from pyspark.sql import Window


In [3]:
if not "spark" in globals() or spark is None:
    print("Creating builder")
    spark = SparkSession.builder.\
            config("spark.executor.memory", "70g").\
            config("spark.driver.memory", "50g").\
            config("spark.memory.offHeap.enabled",True).\
            config("spark.memory.offHeap.size","16g").\
            config("spark.sql.shuffle.partitions", 300).\
            appName('mirexplore').getOrCreate()
    
#spark.conf.set("spark.executor.memory", "70g")
#spark.conf.set("spark.driver.memory", "50g")
#spark.conf.set("spark.memory.offHeap.enabled",True)
#spark.conf.set("spark.memory.offHeap.size","16g")  
#spark.conf.set("spark.sql.shuffle.partitions", 300)

Creating builder


/mnt/extproj/projekte/bartelt/software/miniconda3/envs/mirexplore/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [ ]:
df_int = spark.read.parquet("/mnt/extproj/projekte/textmining/mx_feb24/mx_mirna_gene_consensus_parquet/")

In [ ]:
df_ann = spark.read.parquet("/mnt/extproj/projekte/textmining/mx_feb24/mx_document_annotations/")

In [ ]:
df_int.show(n=5)

In [ ]:
df_ann.show(n=5)

In [ ]:
df_int.filter(col("gene_family") == "CCL2")

In [ ]:
df_int.withColumn("doc_id", explode("evidence_documents")).orderBy("evidence_count", ascending=False).join(other=df_ann, on="doc_id").show()

In [ ]:
df_int.write.parquet("/mnt/extproj/projekte/textmining/mx_feb24/mx_mirna_gene_annotated_parquet", mode="overwrite")

In [ ]:
df_int.printSchema()

In [36]:
import sqlite3

In [37]:
with sqlite3.connect('/mnt/extproj/projekte/textmining/flask-tables/cons_evidences.db') as con:
    cur = con.cursor()
    cur.execute("DROP TABLE IF EXISTS mx;")
    
    ct = """CREATE TABLE "mx" (
    "index" INTEGER PRIMARY KEY AUTOINCREMENT,
    "miRNA_family" TEXT,
    "gene_family" TEXT,
    "interaction" TEXT,
    "organisms" TEXT,
    "evidence_documents" TEXT,
    "sent_evidences" TEXT,
    "evidence_count" INTEGER,
    "sent_count" INTEGER,
    "is_consensus" INTEGER
    )"""
    cur.execute(ct)
    
    pd_df.to_sql(name='mx', con=con, if_exists='append', index_label="index")

In [38]:
con = sqlite3.connect('/mnt/extproj/projekte/textmining/flask-tables/cons_evidences.db')
cur = con.cursor()

In [39]:
a=cur.execute("SELECT sql FROM sqlite_schema WHERE name = 'mx'")
for i in a:
    print(i[0])

CREATE TABLE "mx" (
    "index" INTEGER PRIMARY KEY AUTOINCREMENT,
    "miRNA_family" TEXT,
    "gene_family" TEXT,
    "interaction" TEXT,
    "organisms" TEXT,
    "evidence_documents" TEXT,
    "sent_evidences" TEXT,
    "evidence_count" INTEGER,
    "sent_count" INTEGER,
    "is_consensus" INTEGER
    )


In [40]:
a=cur.execute("SELECT * FROM mx LIMIT 5")
for i in a:
    print(i)

(0, 'miR-6848', 'PLK1', 'GENE_MIR;DOWN', 'hsa;mmu', '34916825', "('34916825.2.13', (48, 52), (197, 212));('34916825.2.13', (48, 52), (201, 212))", 1, 2, 1)
(1, 'miR-6851', 'CEA', 'GENE_MIR;NEU', 'hsa', '36175711', "('36175711.2.8', (303, 306), (144, 155))", 1, 1, 1)
(2, 'miR-6851', 'CEACAM5', 'GENE_MIR;NEU', 'hsa', '36175711', "('36175711.2.8', (303, 306), (144, 155))", 1, 1, 1)
(3, 'miR-6851', 'CEACAM7', 'GENE_MIR;NEU', 'hsa', '36175711', "('36175711.2.8', (303, 306), (144, 155))", 1, 1, 1)
(4, 'miR-6851', 'PSG2', 'GENE_MIR;NEU', 'hsa', '36175711', "('36175711.2.8', (303, 306), (144, 155))", 1, 1, 1)
